# Overview

Welcome to my comprehensive analysis of the Illinois housing market, crafted as part of a resume project to showcase my analytical capabilities. This project aims to provide valuable insights into housing affordability, market volatility, and population trends across Illinois counties.

Leveraging detailed data on price-to-income ratio, new listing percentages, and population figures, I explore key questions that shed light on the economic and demographic landscape of Illinois. Through meticulous calculations and analysis, I identify the most and least affordable counties, examine the volatility in housing listings, and investigate population trends to understand the housing demand in Illinois.

# The Questions

Below are the questions I want to answer in my project:
1. How does affordability vary across Illinois counties?
2. How is Illinois housing inventory listings trending over time?
3. What are the population trends for Illinois counties?

# Tools I Used

For my deep dive into the Illinois housing market, I harnessed the power of several key tools:

* Python: The backbone of my analysis, allowing me to process the data and extract critical insights. I utilized the following Python libraries to enhance my analysis:
    * Pandas Library: Used for comprehensive data analysis and manipulation.
    * Matplotlib Library: Employed to visualize the data, providing clear and informative charts.
    * Seaborn Library: Helped create more advanced and aesthetically pleasing visuals.
    * FRED API: Utilized to access economic data for a richer analysis.
    * Numpy Library: Supported various numerical operations to streamline data processing.
* Jupyter Notebooks: The primary tool for running my Python scripts, enabling me to seamlessly integrate my notes and analysis.

* Visual Studio Code: My go-to environment for executing Python scripts, ensuring efficient coding and debugging.

* Git & GitHub: Essential for version control and sharing my code and analysis, facilitating collaboration and project tracking.

# Data Preparation and Cleanup

This section details the steps taken to prepare the data for analysis, ensuring its accuracy and usability.

### Import Libraries

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import fredapi as fa

### Extract & Clean Data From Illinois Counties 
After importing the necessary libraries and extracting the datasets for 33 counties via a for loop, I perform initial data cleaning tasks to match other datasets.

In [ ]:
fred = fa.Fred(api_key=api_key)

counties_data = pd.read_csv(r'/csv_files/Counties.csv')

# Create an empty list to store DataFrames
dataframes = []

# Loop through each county and fetch data from FRED
for index, row in counties_data.iterrows():
    county_name = row['county_name']
    series_id = row['series_id']

    try:
        data = fred.get_series(series_id)
        data = data.to_frame(name='median_price')
        data['median_price'] = data['median_price'].astype(int) #converts median listing price to int
        data['county_name'] = county_name
        data['year'] = data.index.year
        mask = data['year'].apply(lambda x: x < 2023) #filter data by year to match other datasets
        filtered_data = data[mask]
        dataframes.append(filtered_data)
    except Exception as e:
        print(f"Error retrieving data for {county_name}: {e}")

# Combine DataFrames
combined_df = pd.concat(dataframes)
combined_df = combined_df.reset_index()
combined_df.rename(columns={'index': 'Date'}, inplace=True) #readjust column after resetting


print(combined_df)

# The Analysis
Each Jupyter notebook in this project was tailored to investigate particular aspects of the Illinois housing market. Here’s how I addressed each key question:

## 1. How does affordability vary across Illinois counties?
To measure affordabilty, I calculated the home price to income ratio and grouped the data by county and year. I filtered this data and outputted the top 5 most and least affordable counties. This query highlights how the home price to income ratio varies over time across the selected counties. 

View my notebook with detailed steps here: 2_County_Affordability.




### Visualize Data

In [ ]:
# Combine counties with highest and lowest average pi ratios
combined_affordability = pd.concat([least_affordable_data, most_affordable_data], ignore_index=True)

# Create the line plot
sns.lineplot(x="year", y="pi_ratio", hue="county_name", data=combined_affordability)
plt.xlabel("Year")
plt.ylabel("Home Price-to-Income Ratio")
plt.title("Home Price-to-Income Ratio by County Over Time")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

#### Results

![Counties With Highest & Lowest PIR over Time](./images\Home-PIR-Counties-Over-Time.png)

_Line plot visualizing the Illinois counties with the highest and lowest PIR from 2016 to 2022._

#### Insights:
* The least affordable counties in Illinois require 3-5 times the average income to buy a median-priced home, while the most affordable counties will only take 1.5-2 times the average income. This shows a significant difference between the two groups of counties.
* In Illinois, 4 out of the 5 least affordable counties have shown a gradual decrease in the home price to income ratio from 2016 to 2022. However, 3 out of the 5 most affordable counties have shown a slight increase during the same time period.
* Lake County demonstrates a notable decrease from 2016 to 2022, in contrast to fluctuations in the other counties.

## 2. How is Housing Inventory Listings Trending Over Time?
To examine how new housing listings are trending over time, I aggregated percentage of new listings per county and grouped the data by year. This got me the distribution of new listing percentages for each year. 

View my notebook with detailed steps here: 3_Housing_Inventory.


### Visualize Data

In [ ]:
sns.set_style("whitegrid")
# Create the scatter plot
sns.scatterplot(x='year', y='new_listings', data=anl_df, alpha=0.7)
plt.xlabel('year')
plt.ylabel('New Listings (%)')
plt.title('Average Percentage of New Listings by County Over Time')
sns.despine() 
plt.grid(True)
# Format y axis with %
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, p: f"{y:.0f}%"))
plt.show()

#### Results

![Average Percentage of New Listings by County Over Time](images\Avg-Percentage-New-Listings-Over-Time.png)

_Scatter plot visualizing the percentage of new listings by county from 2017 - mid 2024._

#### Insights:
* There has been a consistent upward trend in the average percentage of new listings, particularly notable from 2022 onward.
* There appears to be a significant surge in new listings in 2024, with one particularly alarming outlier indicating a 40% increase from the previous year.
* The spread of the data points is larger in recent years, indicating increased variability in new listings across counties.   

# What are the population trends in most and least affordable Illinois counties?
To take a look at the population trends, I filtered the data and aggregated each group of counties while excluding an outlier (Cook County). In order to address the significant differences in population, I normalized the values. This will uncover the underlying pattern behind the trends for the most and least affordable Illinois counties. 

View my notebook with detailed steps here: 4_Counties_Population.


### Visualize Data

In [ ]:
def normalize(df):
  return (df - df['population'].min()) / (df['population'].max() - df['population'].min())

normalized_affordable = normalize(mean_population.copy())
normalized_least = normalize(mean_least.copy())

# Plot the normalized data
plt.plot(normalized_affordable['population'], label='Most Affordable Counties')
plt.plot(normalized_least['population'], label='Least Affordable Counties')
plt.legend()
plt.xlabel('Year')
plt.ylabel('Normalized Population')
plt.title('Normalized Population Trends in Illinois')
plt.show()

### Results

![Normalized Population Trends in Illinois](images\Normalized-Population-Trends.png)

_Line plot visualizing the normalized populataion trends from the most and least affordable IL counties from 2016 - 2023._

### Insights:
* Both the most and least affordable IL counties show a downtrend, suggesting that people are leaving these counties. 
* The least affordable counties seem to have experienced volatilty between the years of 2018 - 2022. This can be a sign of inconsistent demand among the least affordable counties. 
* The two groups exhibit opposite trends in the year of 2019 and 2022. This could reflect a poptential relationship among people moving away from less affordable counties into more affordable counties. 

# What I Learned
Throughout this project, I deepened my understanding of the Illinois housing market and enhanced my technical skills in Python, especially in data manipulation and visualization. Here are a few specic things I learned:
* __Advanced Python Usage:__ Utilizing libraries such as Pandas for data manipulation, Seaborn and Matplotlib for data visualization. 
* __Data Cleaning Importance:__ I learned that thorough data cleaning and preparation are crucial before any analysis, ensuring the accuracy and consistency of insights.
* __API Usage__: The projected required extraction of public data available from FRED. Learning how to obtain and manipulate economic indicators has broaden my toolset using available information. 

# Insights
This project provided several general insights into the Illinois housing market. 
* __Home Price to Income Ratio:__ There is an interesting inverse relationship between the counties with the highest and lowest home price to income ratio. The market seems to be contracting, where the signifcantly less affordable counties are becoming slightly more affordable, and the significantly more affordable counties are becoming slightly less affordable. 
* __New Listings Trends:__ There is a clear upward trend in new listings starting in 2022. In 2024 there's a surge in new listings which can signal more volatility in home prices. This is due to more competition in the housing market if demand stays relativeley the same. 
* __Population Trends:__ It's no doubt that the population across these counties in Illinois are decreasing.This shows less demand to reside in these counties, which can possibly lead to volatility in home prices.   

# Challenges I Faced
This project was not without its challenges, but it provided good learning opportunites:
* __Data Inconsistencies:__ Handling inconsistent data entries requires careful consideration and thorough data-cleaning techniques to ensure the integrity of the analysis.
* __Complex Data Visualization:__ Designing effective visual representations of complex datasets was challenging but critical for conveying insights clearly and compellingly.
* __Balancing Depth:__ Deciding how deeply to dive into each analysis while maintaining a broad overview of the data landscape required constant balancing to ensure comprehensive coverage without getting lost in details.

# Conclusion
This analysis of the Illinois housing market has provided valuable insights into the trends shaping the industry. The findings offer a comprehensive understanding of market dynamics, identifying key factors influencing housing availability and affordability. As the market continues to evolve, ongoing analysis will be crucial for making informed decisions.

